# Setup

In [ ]:
import os
import joblib

In [ ]:
online = False

In [ ]:
baseline = True
ABB_ESTIMATE = True

do_sas = baseline
do_stata = baseline
do_ABB = baseline
do_LRT = baseline
do_ConSav = baseline

# Functions to call SAS, Stata and MATLAB

In [ ]:
def run_sas(file):
    
    # a. run
    sasexe = '"C:/Program Files/SASHome/SASFoundation/9.4/sas.exe"'
    out = os.system(f'{sasexe} -sysin {os.getcwd()}/{file}.sas -ICON -NOSPLASH')
    
    # b. check if terminated succesfully
    if out == 0:
        print(f'{file} terminated succesfully')
    else:
        raise Exception(f'{file} did not terminate succesfully')    
        
    # c. clean up
    os.remove(f'{file}.log')        

In [ ]:
def run_stata(file):
    
    # a. run
    stataexe = '"C:\Program Files (x86)\Stata15\StataMP-64.exe"'
    os.system(f'{stataexe} /UseRegistryStartin /e {os.getcwd()}/{file}.do')
    
    # b. check if terminated succesfully
    with open(f'{file}.log') as logfile:
        lastline = list(logfile)[-1]
        if lastline == 'end of do-file\n':
            print(f'{file} terminated succesfully')
        else:
            raise Exception(f'{file} did not terminate succesfully\n {lastline}')
    
    # c. clean up
    os.remove(f'{file}.log')

In [ ]:
def run_matlab(file):
    
    # a. run
    out = os.system('matlab -nosplash -nodesktop -wait -r "run(' + "'" + f'{os.getcwd()}/{file}' + ".m');exit;")
    
    # b. check if terminated succesfully
    if out == 0:
        print(f'{file} terminated succesfully')
    else:
        raise Exception(f'{file} did not terminate succesfully')    

In [ ]:
def run_jupyter(file):
    
    os.system(f'jupyter nbconvert --to notebook --execute --ExecutePreprocessor.timeout=-1 --inplace "{file}.ipynb"')
    

In [ ]:
def run_jupyter_parallel(files):
    
    tasks = (joblib.delayed(run_jupyter)(ipynbfile) for ipynbfile,do in ipynbfiles if do)
    joblib.Parallel(n_jobs=len(files))(tasks)


# Fetch data and select sample(s)

**Inputs:**

1. Danish register data (INDUPD, BEF, FAIN, IDAP, with views located in H:/Rawdata/706248/views/)
2. Consumer price data: data/PRIS61.xlsx (from Statistikbanken.dk)
3. House price data: data/EJEN6.xlsx (from Statistikbanken.dk)

## Run SAS

In [ ]:
# a. setup
sasfiles = [('0_1_import_datasets',True),
            ('0_2_merge_datasets',True),
            ('0_3_export_datasets',True)]

# b. run
if do_sas and online:
    [run_sas(sasfile) for sasfile,do in sasfiles if do]

**Output:** data/sample_raw.dta

## Run STATA

In [ ]:
# a. setup
statafiles = [('1_1_construct_deflator',True),
              ('1_2_construct_deflator_house',True),
              ('1_3_deflate',True),
              ('1_4_export_csv',True)]

# b. run
if do_stata and online:
    [run_stata(statafile) for statafile,do in statafiles if do]

**Output:**

1. data/logY_SAMPLE.csv
2, data/logY_raw_SAMPLE.csv
3. data/birthyear_SAMPLE.csv
4. data/a_mean_SAMPLE.csv
5. data/a_p50_SAMPLE.csv
6. data/Y_zerocutoff_SAMPLE.csv
7. data/logY_unbalanced_SAMPLE.csv

**Results:**

1. output/units.xls
2. output/totobs.xls
3. output/units_zerocutoff.xls
4. output/totobs_zerocutoff.xls
5. output/units_unbalanced.xls
6. output/totobs_unbalanced.xls

# Replication

In [ ]:
if not online:
    run_jupyter('replication')

# Arellano-Blundell-Bonhomme

In [ ]:
# a. setup
mfiles = [('ABB/run_estimate',ABB_ESTIMATE), 
          ('ABB/run_simulate',True)]

# b. run
if do_ABB:
    if online:
        [run_matlab(mfile) for mfile,do in mfiles if do]
    else:
        run_matlab('ABB/run_simulate')

**Output:** data/abb_sim.csv

# LRT

In [ ]:
# a. setup
ipynbfiles = [('2_1_main_results',True),
              ('2_2_cross_validation',True),
              ('2_3_attrition',True),
              ('2_4_unbalanced',True),
              ('2_5_levels',True),
              ('2_6_cohorts',True)]

# b. run
if do_LRT:
    run_jupyter_parallel(ipynbfiles)
    
# c. export
if do_LRT and online:
    run_jupyter('2_7_censor_data_and_export')

**Output:**

1. data/mean_logY.txt.txt
2. data/mean_Y.txt
3. data/mean_Y_lev.txt
4. data/num_leafs.txt
5. data/prob_G_ini.txt
6. data/var_logY.txt
7. data/ypred_G_t*.txt
8. data/tran_t*.txt
9. censored_estimates/*

**Results:**

1. output/PT_estimates.txt
2. output/*.pdf

# Consumption-saving model

In [ ]:
# a. setup
mfiles = [('ConSav/run',True)]

# b. run
if do_ConSav:
    [run_matlab(mfile) for mfile,do in mfiles if do]

**Results:**

1. output/ConSav/main_table.tex
2. output/ConSav/*.pdf
3. output/ConSavEstimates/*.mat